In [1]:
import os
import requests
import json
import sys
import time
import pathlib  # Added pathlib
from requests.auth import HTTPBasicAuth

In [5]:
def read_geom(file_path):
    with open(file_path) as f:
        bbox = json.load(f)['features'][0]['geometry']
        return bbox
    
def place_order(request, auth):
    response = requests.post('https://api.planet.com/compute/ops/orders/v2', data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = 'https://api.planet.com/compute/ops/orders/v2' + '/' + order_id
    return order_url

def poll_for_success(order_url, auth):
    while True:
        r = requests.get(order_url, auth=(api_key, ""))
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        # time.sleep(10)


def download_results(results, geojson_file, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    # Extract the parking lot number from the GeoJSON file name
    parking_lot_number = os.path.splitext(geojson_file)[0]
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', 'Ocean_City', parking_lot_number, name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            path.parent.mkdir(parents=True, exist_ok=True)  # Create the directory if it doesn't exist
            r = requests.get(url, allow_redirects=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))


In [2]:
geojson_folder = '/home/idisc02/Desktop/i2sc/detect-car-LR/data/Ocean_City'

